# 3D Visualization with `atoms_viewer`

`atoms_viewer` is a small Python wrapper for the JavaScript library [3Dmol.js](https://3dmol.csb.pitt.edu/index.html).
With this wrapper, you can create custom 3D visualizations of proteins, molecules, and materials within a Jupyter notebook.
In addition, you can download snapshots of your visualizations and generate animated PNGs.
For more details, check out the demo below.

On a high level, this wrapper allows you to call JavaScript functions from within your Jupyter notebook to manipulate the `3Dmol.js` viewer object `GLViewer` documented [here](https://3dmol.csb.pitt.edu/doc/\$3Dmol.GLViewer.html). 
In particular, the `queue` method (see below), allows you to call any method of the `GLViewer` [object](https://3dmol.csb.pitt.edu/doc/\$3Dmol.GLViewer.html).

First, let's load the package. This step loads all required JavaScript libraries.

In [ ]:
import atoms_viewer

from atoms_viewer import AtomsViewer

Don't forget to run `init()` to initialize the library.

In [ ]:
atoms_viewer.init()

## Getting Started

Let's load the PDB file of a small protein called crambin.

In [ ]:
with open("resources/3nir.pdb", 'r') as f:
    pdb = f.read()

Next, we can visualize the protein in a "cartoon"-sh style.

In [ ]:
# Instantiating AtomsViewer object
pv = AtomsViewer()

# Configure canvas
pv.create(style={'width': '500px', 'height': '500px'}, config={'backgroundAlpha': 1.0})

# Add model by passing the PDB file content and a format specification
pv.queue('addModel', pdb, 'pdb')

# Set the visualization style. To select all atoms, put an emptry dictionary as the first argument
pv.queue('setStyle', {}, {'cartoon': {'color': 'spectrum'}})

# Put protein into focus
pv.queue('zoomTo')

# Don't forget to tell the library to render your scene
pv.queue('render')

# Process queue of "statements"
pv.process()

You can update the scene later.
For instance, we can add a Van der Waals surface with the [addSurface](https://3dmol.csb.pitt.edu/doc/$3Dmol.GLViewer.html#addSurface) function.

In [ ]:
# Surface=1 is the vdW surface
pv.queue('addSurface', 1, {'opacity': 0.7, 'color': 'white'})
pv.queue('render')
code = pv.process(debug=True)

To see how that worked, let's have a look at the generated code.

In [ ]:
print(code)

Once your happy with the visualization and orientation, you can save it as a PNG.

In [ ]:
pv.download_png('3nir.png')
pv.process()

## Crystals

With `3Dmol.js` you can also visualize crystal structures.

In [ ]:
import ase.io
import io

cv = AtomsViewer()
cv.create()

with open('resources/gen_15_Li1_Ag1_S2_0.cif') as f:
    cv.queue('addModel', f.read(), 'cif')

cv.queue('addUnitCell')
cv.queue('setStyle', {}, {'sphere': {'radius': 0.5}})

cv.queue('zoomTo')
cv.queue('render')
cv.process()

## Interaction with IPython Widgets

Load a set of trajectories from the 3BPA dataset.

In [ ]:
atoms_list = ase.io.read('resources/3bpa.xyz', index=':', format='extxyz')

You can easily manipulate the scene with IPython widgets.
For demonstration purposes, we colored the first and second atom purple.

In [ ]:
import ipywidgets
from IPython.display import display

# Create viewer
av = AtomsViewer()
av.create()
av.process()

# Create GUI
slider = ipywidgets.IntSlider(value=0, min=0, max=len(atoms_list) - 1, step=1, description='Index:')
check_box1 = ipywidgets.Checkbox(value=False, description='Show forces', indent=True)
check_box2 = ipywidgets.Checkbox(value=False, description='Show labels', indent=True)
ui = ipywidgets.VBox([slider, check_box1, check_box2])

def callback(idx: int, show_forces: bool, show_labels: bool) -> None:
    av.remove_all()
    
    f = io.StringIO()
    ase.io.write(f, atoms_list[idx], format='xyz')
    av.queue('addModel', f.getvalue(), 'xyz')
    
    av.queue('setStyle', {}, {'stick': {}})
    av.queue('addStyle', {}, {'sphere': {'scale': 0.3}})
    
    # Color 0th and 1st atom
    av.queue('addStyle', {'serial': [0, 1]}, {'sphere': {'color': 'purple'}})

    if show_forces:
        av.add_arrows(positions=atoms_list[idx].arrays['positions'], vectors=0.9 * atoms_list[idx].arrays['forces'])
    
    if show_labels:
        for i in range(len(atoms_list[idx])):
            av.queue('addLabel', str(i), {}, {'serial': i})
    
    av.queue('render')
    av.process()

out = ipywidgets.interactive_output(callback, {'idx': slider, 'show_forces': check_box1, 'show_labels': check_box2})
av.queue('zoomTo')
av.process()
display(ui, out)

## Generating Animated PNGs

With the `take_snapshot` and `download_apng` function, one can generate animated PNG (APNG) files.

In [ ]:
anv = AtomsViewer()
anv.create(style={'width': '500px', 'height': '500px'})
anv.queue('rotate', 45, {'x':1, 'y': 1, 'z': 1});

for idx in range(len(atoms_list)):
    anv.remove_all()
    with io.StringIO() as f:
        ase.io.write(f, atoms_list[idx], format='xyz')
        anv.queue('addModel', f.getvalue(), 'xyz')
    
    anv.queue('setStyle', {}, {'stick': {}})
    anv.queue('addStyle', {}, {'sphere': {'scale': 0.3}})
    anv.add_arrows(positions=atoms_list[idx].arrays['positions'], vectors=0.9 * atoms_list[idx].arrays['forces'])
    
    anv.queue('render')
    if idx == 0:
        anv.queue('zoomTo')
        
    anv.take_snapshot()
    anv.process()

anv.download_apng('animated.png', delay=2000)
anv.process()

Open the generated file with your browser or simply include it here to the atoms wiggle.
with e.g.,

`<img src="<PATH>/animated.png" style="height:200px;width:200px" />`

## Debugging

To better understand what is going on underneath the hood, pass `debug=True` to the process method. This way, the functions returns the HTML code that is passed to the IPython session. Together with the `dry_run=True` option, you ensure that the code is not actually being run.

In [ ]:
pv.remove_all()
pv.queue('render')
print(pv.process(debug=True, dry_run=True))